# Web Scraping Project

We are going to take out information from a web page using BeautifulSoup library. We will extract cricket data that is available in `espncricinfo.com`.

We have two webpages to start with:
1. Most runs scored in ICC Cricket World Cups by players. This webpage is having only one page of entries. The [link](https://stats.espncricinfo.com/ci/engine/records/batting/most_runs_career.html?id=12;type=trophy) is here.
2. Batting record in One Day International matches of all players. This information is divided into multiple pages and in this task, we will get information from all pages. The [link](https://stats.espncricinfo.com/ci/engine/stats/index.html?class=2;template=results;type=batting) is here.

In [1]:
# Importing relevant libraries
from bs4 import BeautifulSoup
import requests
import itertools
import pandas as pd
import re

In [2]:
# Assigning the webpage url to a variable
url = 'https://stats.espncricinfo.com/ci/engine/records/batting/most_runs_career.html?id=12;type=trophy'

# Checking status code
status = requests.get(url)
status

<Response [200]>

In [3]:
# Getting all html code from url
text = requests.get(url).text

In [4]:
# Creating a BeautifulSoup object
soup = BeautifulSoup(text, 'lxml')

In [5]:
# Getting table tag at 0th index
table = soup.find_all('table', class_='engineTable')[0]

# Getting column names
column_names = [i.text for i in table.find_all('th')]

column_names

['Player',
 'Span',
 'Mat',
 'Inns',
 'NO',
 'Runs',
 'HS',
 'Ave',
 'BF',
 'SR',
 '100',
 '50',
 '0',
 '4s',
 '6s']

In [6]:
# Getting tbody tag from table
body = table.find('tbody')

# Getting all the information about players
data = []
rows = body.find_all('tr')
for row in rows:
    content = []
    for ls in row.find_all('td'):
        content.append(ls.text)
    data.append(content)

In [7]:
# Creating a dataframe using data and column_names
most_runs_wc = pd.DataFrame(data=data, columns=column_names)
most_runs_wc.head()

,Player,Span,Mat,Inns,NO,Runs,HS,Ave,BF,SR,100,50,0,4s,6s
0,SR Tendulkar (INDIA),1992-2011,45,44,4,2278,152,56.95,2560,88.98,6,15,2,241,27
1,RT Ponting (AUS),1996-2011,46,42,4,1743,140*,45.86,2180,79.95,5,6,1,145,31
2,KC Sangakkara (SL),2003-2015,37,35,8,1532,124,56.74,1770,86.55,5,7,1,147,14
3,BC Lara (WI),1992-2007,34,33,4,1225,116,42.24,1420,86.26,2,7,1,131,17
4,AB de Villiers (SA),2007-2015,23,22,3,1207,162*,63.52,1029,117.29,4,6,4,121,37


We have got the information we wanted from the webpage and we can go ahead with data cleaning to make the data ready for analysis.

In [8]:
# Extracting Country information and removing unnecessary info from Player column
most_runs_wc['Country'] = most_runs_wc['Player'].str.extract(r'\((\w+)\)')
most_runs_wc['Player'] = most_runs_wc['Player'].str.extract(r'([\w\s]+)\s\(')
most_runs_wc.head()

,Player,Span,Mat,Inns,NO,Runs,HS,Ave,BF,SR,100,50,0,4s,6s,Country
0,SR Tendulkar,1992-2011,45,44,4,2278,152,56.95,2560,88.98,6,15,2,241,27,INDIA
1,RT Ponting,1996-2011,46,42,4,1743,140*,45.86,2180,79.95,5,6,1,145,31,AUS
2,KC Sangakkara,2003-2015,37,35,8,1532,124,56.74,1770,86.55,5,7,1,147,14,SL
3,BC Lara,1992-2007,34,33,4,1225,116,42.24,1420,86.26,2,7,1,131,17,WI
4,AB de Villiers,2007-2015,23,22,3,1207,162*,63.52,1029,117.29,4,6,4,121,37,SA


In [9]:
# Checking shape
most_runs_wc.shape

(50, 16)

In [10]:
# Creating a new column for batsman being Not Out on his High Score
most_runs_wc['Not_Out_on_High_Score'] = most_runs_wc['HS'].str.contains(r'\*')

# Removing any non-digit character
most_runs_wc['HS'] = most_runs_wc['HS'].str.replace('\*','')
most_runs_wc.head()

,Player,Span,Mat,Inns,NO,Runs,HS,Ave,BF,SR,100,50,0,4s,6s,Country,Not_Out_on_High_Score
0,SR Tendulkar,1992-2011,45,44,4,2278,152,56.95,2560,88.98,6,15,2,241,27,INDIA,False
1,RT Ponting,1996-2011,46,42,4,1743,140,45.86,2180,79.95,5,6,1,145,31,AUS,True
2,KC Sangakkara,2003-2015,37,35,8,1532,124,56.74,1770,86.55,5,7,1,147,14,SL,False
3,BC Lara,1992-2007,34,33,4,1225,116,42.24,1420,86.26,2,7,1,131,17,WI,False
4,AB de Villiers,2007-2015,23,22,3,1207,162,63.52,1029,117.29,4,6,4,121,37,SA,True


In [11]:
# Checking for missing values and datatypes
most_runs_wc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 17 columns):
Player                   50 non-null object
Span                     50 non-null object
Mat                      50 non-null object
Inns                     50 non-null object
NO                       50 non-null object
Runs                     50 non-null object
HS                       50 non-null object
Ave                      50 non-null object
BF                       50 non-null object
SR                       50 non-null object
100                      50 non-null object
50                       50 non-null object
0                        50 non-null object
4s                       50 non-null object
6s                       50 non-null object
Country                  50 non-null object
Not_Out_on_High_Score    50 non-null bool
dtypes: bool(1), object(16)
memory usage: 6.4+ KB


We will have to change most columns data type to `int` of `float` based on the values.

In [12]:
# Removing non-digit characters
most_runs_wc['4s'] = most_runs_wc['4s'].str.replace('+','')
most_runs_wc['6s'] = most_runs_wc['6s'].str.replace('+','')

# Changing datatype of columns
int_columns = ['Mat','Inns','NO','Runs','HS','BF','100','50','0','4s','6s']
float_columns = ['Ave','SR']

for i in int_columns:
    most_runs_wc[i] = most_runs_wc[i].astype(int)
    
for f in float_columns:
    most_runs_wc[f] = most_runs_wc[f].astype(float)

In [13]:
# Verifying the change
most_runs_wc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 17 columns):
Player                   50 non-null object
Span                     50 non-null object
Mat                      50 non-null int32
Inns                     50 non-null int32
NO                       50 non-null int32
Runs                     50 non-null int32
HS                       50 non-null int32
Ave                      50 non-null float64
BF                       50 non-null int32
SR                       50 non-null float64
100                      50 non-null int32
50                       50 non-null int32
0                        50 non-null int32
4s                       50 non-null int32
6s                       50 non-null int32
Country                  50 non-null object
Not_Out_on_High_Score    50 non-null bool
dtypes: bool(1), float64(2), int32(11), object(3)
memory usage: 4.2+ KB


In [14]:
# Changing position of columns
most_runs_wc = most_runs_wc.reindex(columns=['Player', 'Country', 'Span', 'Mat', 'Inns', 'NO', 
                                             'Runs', 'HS', 'Not_Out_on_High_Score', 'Ave', 'BF', 'SR',
                                             '100', '50', '0', '4s', '6s'])

most_runs_wc.head()

,Player,Country,Span,Mat,Inns,NO,Runs,HS,Not_Out_on_High_Score,Ave,BF,SR,100,50,0,4s,6s
0,SR Tendulkar,INDIA,1992-2011,45,44,4,2278,152,False,56.95,2560,88.98,6,15,2,241,27
1,RT Ponting,AUS,1996-2011,46,42,4,1743,140,True,45.86,2180,79.95,5,6,1,145,31
2,KC Sangakkara,SL,2003-2015,37,35,8,1532,124,False,56.74,1770,86.55,5,7,1,147,14
3,BC Lara,WI,1992-2007,34,33,4,1225,116,False,42.24,1420,86.26,2,7,1,131,17
4,AB de Villiers,SA,2007-2015,23,22,3,1207,162,True,63.52,1029,117.29,4,6,4,121,37


In [15]:
# Renaming columns for better data knowledge
most_runs_wc.rename(columns={'Span':'Career', 
                             'Mat': 'Matches',
                             'Inns': 'Innings',
                             'NO': 'Not_Outs',
                             'HS': 'High_Score',
                             'Ave':'Average',
                             'BF': 'Balls_Faced',
                             'SR': 'Strike_Rate',
                             '100': 'Centuries',
                             '50': 'Half_Centuries',
                             '0': 'Ducks'}, inplace=True)

most_runs_wc.head()

,Player,Country,Career,Matches,Innings,Not_Outs,Runs,High_Score,Not_Out_on_High_Score,Average,Balls_Faced,Strike_Rate,Centuries,Half_Centuries,Ducks,4s,6s
0,SR Tendulkar,INDIA,1992-2011,45,44,4,2278,152,False,56.95,2560,88.98,6,15,2,241,27
1,RT Ponting,AUS,1996-2011,46,42,4,1743,140,True,45.86,2180,79.95,5,6,1,145,31
2,KC Sangakkara,SL,2003-2015,37,35,8,1532,124,False,56.74,1770,86.55,5,7,1,147,14
3,BC Lara,WI,1992-2007,34,33,4,1225,116,False,42.24,1420,86.26,2,7,1,131,17
4,AB de Villiers,SA,2007-2015,23,22,3,1207,162,True,63.52,1029,117.29,4,6,4,121,37


## Task 2

In [16]:
# Creating url_2 varialbe
url_2 = 'https://stats.espncricinfo.com/ci/engine/stats/index.html?class=2;template=results;type=batting'

# Checking request status
status = requests.get(url_2)
status

<Response [200]>

In [17]:
# Getting text data from url_2
text2 = requests.get(url_2).text

In [18]:
# Creating a BeautifulSoup object
soup2 = BeautifulSoup(text2, 'lxml')

In [19]:
# Getting table tag
table2 = soup2.find_all('table')[2]

In [20]:
# Getting all column names
column_names2 = [i.text for i in table2.find_all('th')]

# Removing last column_name which doesn't have any information
column_names2 = column_names2[:-1]
column_names2

['Player',
 'Span',
 'Mat',
 'Inns',
 'NO',
 'Runs',
 'HS',
 'Ave',
 'BF',
 'SR',
 '100',
 '50',
 '0']

In [21]:
# Creating a list of lists of data
rows2 = table2.tbody.find_all('tr')
data2 = []

for row in rows2:
    content = []
    for ls in row.find_all('td')[:-1]:
        content.append(ls.text)
        
    data2.append(content)

The data we got till now is actually a list of lists that we created for the first page only, but upon checking, we can see that there are 53 pages in total. We will have to find a way to get data from all the pages.

The link of page 1 : https://stats.espncricinfo.com/ci/engine/stats/index.html?class=2;template=results;type=batting

The link of page 2 : https://stats.espncricinfo.com/ci/engine/stats/index.html?class=2;page=2;template=results;type=batting

These links suggest that only difference between these links is `page=2;`, which is expected to go till 53. So we can use a for loop to replace this value with every iteration and get the data appended to `data2`.

In [22]:
# Iterating through urls and getting all data
page_no = 2
for i in range(52):
    url_new = url_2.replace(r'2;', '2;page=' + str(page_no) + ';')
    
    text = requests.get(url_new).text
    soup = BeautifulSoup(text, 'lxml')
    table = soup.find_all('table')[2]
    rows = table.tbody.find_all('tr')
    for row in rows:
        content = []
        for ls in row.find_all('td')[:-1]:
            content.append(ls.text)
            
        data2.append(content)
    page_no += 1

In [23]:
# Making a pandas dataframe from data2 list
odi_batting = pd.DataFrame(data=data2, columns=column_names2)
odi_batting.head()

,Player,Span,Mat,Inns,NO,Runs,HS,Ave,BF,SR,100,50,0
0,SR Tendulkar (INDIA),1989-2012,463,452,41,18426,200*,44.83,21368,86.23,49,96,20
1,KC Sangakkara (Asia/ICC/SL),2000-2015,404,380,41,14234,169,41.98,18048,78.86,25,93,15
2,RT Ponting (AUS/ICC),1995-2012,375,365,39,13704,164,42.03,17046,80.39,30,82,20
3,ST Jayasuriya (Asia/SL),1989-2011,445,433,18,13430,189,32.36,14725,91.20,28,68,34
4,DPMD Jayawardene (Asia/SL),1998-2015,448,418,39,12650,144,33.37,16020,78.96,19,77,28


In [24]:
# Checking shape
odi_batting.shape

(2648, 13)

In [25]:
# Getting data on missing values and datatype of columns
odi_batting.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2648 entries, 0 to 2647
Data columns (total 13 columns):
Player    2648 non-null object
Span      2648 non-null object
Mat       2648 non-null object
Inns      2648 non-null object
NO        2648 non-null object
Runs      2648 non-null object
HS        2648 non-null object
Ave       2648 non-null object
BF        2648 non-null object
SR        2648 non-null object
100       2648 non-null object
50        2648 non-null object
0         2648 non-null object
dtypes: object(13)
memory usage: 269.0+ KB


In [26]:
# Creating Country column of each player
odi_batting['Country'] = odi_batting['Player'].str.split().str[-1].str.replace('(','').str.replace(')','')
odi_batting.head()

,Player,Span,Mat,Inns,NO,Runs,HS,Ave,BF,SR,100,50,0,Country
0,SR Tendulkar (INDIA),1989-2012,463,452,41,18426,200*,44.83,21368,86.23,49,96,20,INDIA
1,KC Sangakkara (Asia/ICC/SL),2000-2015,404,380,41,14234,169,41.98,18048,78.86,25,93,15,Asia/ICC/SL
2,RT Ponting (AUS/ICC),1995-2012,375,365,39,13704,164,42.03,17046,80.39,30,82,20,AUS/ICC
3,ST Jayasuriya (Asia/SL),1989-2011,445,433,18,13430,189,32.36,14725,91.20,28,68,34,Asia/SL
4,DPMD Jayawardene (Asia/SL),1998-2015,448,418,39,12650,144,33.37,16020,78.96,19,77,28,Asia/SL


In [27]:
# Creating a function to correctly display country name
def correcting_country(x):
    
    '''This function will take in a value and then check for any continent, cricket governing body, non-word characters
    and then replace them with empty string so that Country column only displays country or countries that player
    has played for.'''
    
    to_remove = ['Afr', 'ICC', 'Asia']
    count = 0
    if '/' in x:
        for i in to_remove:
            if i not in x:
                count += 1
        if count == 3:
            x = x.replace('/','-')
            return x.upper()
    if '/' in x:
        for i in to_remove:
            if i in x:
                x = x.replace(i, '')
        x = x.replace('/','')
        return x.upper()
    else:
        return x.upper()
    
odi_batting['Country'] = odi_batting['Country'].apply(correcting_country)
odi_batting.head()

,Player,Span,Mat,Inns,NO,Runs,HS,Ave,BF,SR,100,50,0,Country
0,SR Tendulkar (INDIA),1989-2012,463,452,41,18426,200*,44.83,21368,86.23,49,96,20,INDIA
1,KC Sangakkara (Asia/ICC/SL),2000-2015,404,380,41,14234,169,41.98,18048,78.86,25,93,15,SL
2,RT Ponting (AUS/ICC),1995-2012,375,365,39,13704,164,42.03,17046,80.39,30,82,20,AUS
3,ST Jayasuriya (Asia/SL),1989-2011,445,433,18,13430,189,32.36,14725,91.20,28,68,34,SL
4,DPMD Jayawardene (Asia/SL),1998-2015,448,418,39,12650,144,33.37,16020,78.96,19,77,28,SL


In [28]:
# Removing anything other than Player name from Player column
odi_batting['Player'] = odi_batting['Player'].str.extract(r'([\w\s]+)\s\(')

odi_batting.head()

,Player,Span,Mat,Inns,NO,Runs,HS,Ave,BF,SR,100,50,0,Country
0,SR Tendulkar,1989-2012,463,452,41,18426,200*,44.83,21368,86.23,49,96,20,INDIA
1,KC Sangakkara,2000-2015,404,380,41,14234,169,41.98,18048,78.86,25,93,15,SL
2,RT Ponting,1995-2012,375,365,39,13704,164,42.03,17046,80.39,30,82,20,AUS
3,ST Jayasuriya,1989-2011,445,433,18,13430,189,32.36,14725,91.20,28,68,34,SL
4,DPMD Jayawardene,1998-2015,448,418,39,12650,144,33.37,16020,78.96,19,77,28,SL


In [29]:
# Replacing unnecessary characters from odi_batting dataframe
odi_batting = odi_batting.applymap(lambda x: re.sub('[^\w\.\-\s]', '', x))

# Creating a new column for batsman being Not Out on his highest score
odi_batting['Not_Out_on_High_Score'] = odi_batting['HS'].str.contains('\*')

In [30]:
# Checking last 5 rows
odi_batting.tail()

,Player,Span,Mat,Inns,NO,Runs,HS,Ave,BF,SR,100,50,0,Country,Not_Out_on_High_Score
2643,Washington Sundar,2017-2017,1,-,-,-,-,-,-,-,-,-,-,INDIA,False
2644,M Watkinson,1996-1996,1,-,-,-,-,-,-,-,-,-,-,ENG,False
2645,S Weerakoon,2012-2012,2,-,-,-,-,-,-,-,-,-,-,SL,False
2646,Zahir Khan,2019-2019,1,-,-,-,-,-,-,-,-,-,-,AFG,False
2647,Zakir Hasan,1997-1997,1,-,-,-,-,-,-,-,-,-,-,BDESH,False


In [31]:
# Replacing '-' from columns that shouldn't have it
replace_in = odi_batting.columns[3:-1]

for c in replace_in:
    odi_batting[c] = odi_batting[c].replace('-','0')

In [32]:
# Changing datatype of columns
int_c = ['Mat','Inns','NO','Runs','HS','BF','100','50','0']
float_c = ['Ave','SR']

for i in int_c:
    odi_batting[i] = odi_batting[i].astype(int)
    
for f in float_c:
    odi_batting[f] = odi_batting[f].astype(float)

In [33]:
# Checking updated datatypes
odi_batting.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2648 entries, 0 to 2647
Data columns (total 15 columns):
Player                   2648 non-null object
Span                     2648 non-null object
Mat                      2648 non-null int32
Inns                     2648 non-null int32
NO                       2648 non-null int32
Runs                     2648 non-null int32
HS                       2648 non-null int32
Ave                      2648 non-null float64
BF                       2648 non-null int32
SR                       2648 non-null float64
100                      2648 non-null int32
50                       2648 non-null int32
0                        2648 non-null int32
Country                  2648 non-null object
Not_Out_on_High_Score    2648 non-null bool
dtypes: bool(1), float64(2), int32(9), object(3)
memory usage: 199.2+ KB


In [34]:
# Replacing column names with suitable names
odi_batting.rename(columns={'Span':'Career',
                           'Mat':'Matches',
                           'Inns':'Innings',
                           'NO':'Not_Outs',
                           'HS':'High_Score',
                           'Ave':'Average',
                           'BF':'Balls_Faced',
                           'SR':'Strike_Rate',
                           '100':'Centuries',
                           '50':'Half_Centuries',
                           '0':'Ducks'}, inplace=True)

In [35]:
# Reindexing on columns basis
odi_batting = odi_batting.reindex(columns=['Player','Country','Career','Matches','Innings','Not_Outs','Runs',
                                           'High_Score','Not_Out_on_High_Score','Average','Balls_Faced',
                                           'Strike_Rate','Centuries','Half_Centuries','Ducks'])

odi_batting.head()

,Player,Country,Career,Matches,Innings,Not_Outs,Runs,High_Score,Not_Out_on_High_Score,Average,Balls_Faced,Strike_Rate,Centuries,Half_Centuries,Ducks
0,SR Tendulkar,INDIA,1989-2012,463,452,41,18426,200,False,44.83,21368,86.23,49,96,20
1,KC Sangakkara,SL,2000-2015,404,380,41,14234,169,False,41.98,18048,78.86,25,93,15
2,RT Ponting,AUS,1995-2012,375,365,39,13704,164,False,42.03,17046,80.39,30,82,20
3,ST Jayasuriya,SL,1989-2011,445,433,18,13430,189,False,32.36,14725,91.20,28,68,34
4,DPMD Jayawardene,SL,1998-2015,448,418,39,12650,144,False,33.37,16020,78.96,19,77,28
